# Homework 1 - Streaming (15 pts)

### DUE: 02/18/2020 at 5:30pm

This homework is for practicing Python’s *generators* and the design of *streaming* algorithms in general. We’re going to use the **taxi_events.csv** and **taxi_trips.csv** data sets. Both of them are available on NYU Classes under *Data Sets* section. You are required to turn in this notebook with all the parts filled in place of <###>. Your notebook must be named BDM\_HW1\_NetID.ipynb.

## Task 1 (8 points)

Your task is to **compute the maximum number of active taxi cabs** that were hired at any point in time. This the same as computing the maximum number of taxi cabs that have passengers. The input data set is **taxi_events.csv**, which logged all pick-up and drop-off events for all taxi trips. The description of the fields in this file is as follows:

|Column name|Description|
|--|--|
|time |The timestamp of the event. All events are sorted increasingly by their timestamps. |
|vehicle_id |The unique ID of the taxi vehicle involved in this event. |
|event |A string of either *"pickup"* or *"dropoff"* for describing the drop-off or pick-up event, respectively. |

For example, let's assume that on *Feb-01-2015*, there was a taxi that picked up a passenger at midnight and dropped her off at 14 minutes past midnight. If the taxi cab has the Vehicle ID of *V102*, then you should see two events being logged in this data set as:

<pre>
...
2015-02-01 00:00:00+00,V102,pickup
...
2015-02-01 00:14:00+00,V102,dropoff
...
</pre>

You are given the above data set in a streaming fashion (reading in row by row), and must design a streaming algorithm that uses the least possible additional memory to compute the maximum number of hired taxi cabs at any point in time. Again, this is equivalent to having a virtual dispatcher, who repeatedly ask every second *"how many taxis are being hired (having passengers) at the moment?"*, and then log the maximum number during the entire period.

Please modify the code snippet below to complete this task. Your code should only output a single number, which is the maximum number of hired taxi cabs. Of course, you can add additional initialization codes outside of the for loop as needed. Additional, please provide a brief rationale and/or justification for your design after the code.

In [4]:
#%%time
import csv

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row

maxHiredCabs = 0
CurrentHiredCabs = 0
taxi_count = {}

for row in csvRows('taxi_events.csv'):
    #<YOUR STREAMING CODE HERE> 
    vehicle_id = row['vehicle_id']
    if row['event']=='dropoff':
        taxi_count[vehicle_id] = taxi_count[vehicle_id] - 1
        if taxi_count[vehicle_id] == 0:
            taxi_count.pop(vehicle_id)
    elif row['event']=='pickup':
        taxi_count[vehicle_id] = taxi_count.get(vehicle_id, 0) + 1
        CurrentHiredCabs = len(taxi_count)
        if CurrentHiredCabs > maxHiredCabs:
            maxHiredCabs = CurrentHiredCabs

print(maxHiredCabs)



250


#### RATIONALE AND JUSTIFICATION

By obseerving the datset, I found that carpooling exists in this dataset, that a taxi might carry several batches of customers at the same time, a case thet basically complicate the situation. Cosidering this, I create a dictionary with the keys of vehicle id and the values of the number of the number of customer the taxi is carrying.

1) If a pickup record comes, we search the vehicle id in the dictionary and create a new key-value pair if the vehicle id has never appeared in the dictionary. If vihicle id has been there, add the according value by 1. By count the length of the dictionary we can know how many taxies are in service at a particular time. If the current number of taxi is larger than the maxinum number, we should assign the current number to the maxinum number. 

2) If a dropoff record comes, we search the vehicle id in the dictionary and subtract the according value by 1. If the value is equal to zero after subtracting, which means that this taxi is not in service, we should delete the vehicle id from the dictionary at all. 

## Task 2 (7 points)

The objective of this task is identical to Task 1's but you are asked to use the full **taxi_trips.csv** data set instead of the events. The main difference (and challenge) is that both pick-up and drop-off event for each trip is now presented in a single record, thus, the drop-off events are not sorted by their timestamps. You are again asked to do this in a streaming fashion that needs to minimize the amount of memory usage. Please modify the code below accordingly, and also with a brief explaination of the solution.

Below is the description of the **taxi_trips.csv** file, which is sorted only by the pick-up time:

|Column name|Description|
|--|--|
|trip_duration |The duration of the trip in seconds. This field is for your convenience since it can be derived also from the pick-up and drop-off times. |
|pickup_time |The timestamp of the pick-up of the trip. All trip records are sorted increasingly by their pick-up times. |
|dropoff_time |The timestamp of the drop-off event. |
|vehicle_id |The unique ID of the taxi vehicle involved in this trip record. |



In [5]:
#%%time
import csv

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row

maxHiredCabs = 0
CurrentHiredCabs = 0
droptime_vihid = []
current_time = '2015-02-01 00:00:00+00'
for row in csvRows('taxi_trips.csv'):
    #<YOUR STREAMING CODE HERE>
    last_time = current_time
    current_time = row['pickup_time'] 
    vehicle_id = row['vehicle_id']
    if current_time > last_time:
        droptime_vihid = [time_id for time_id in droptime_vihid if time_id[0] > current_time]
    droptime_vihid.append((row['dropoff_time'], vehicle_id))
    _, vihid = zip(*droptime_vihid)
    CurrentHiredCabs = len(set(vihid))
    if CurrentHiredCabs > maxHiredCabs:
        maxHiredCabs = CurrentHiredCabs

    
print(maxHiredCabs)

250


#### RATIONALE AND JUSTIFICATION

Since the dropoff time is not sorted by time, we should cache the dropoff time as well as vehicle id. I use a list of tuples (vehicle id, dropoff time) to store the information in the streaming task. 

When a record comes, I compare the current timestamp with all the timestamp in the list and delete the tuples with timestamp before the current timestamp. This makes sure the list only caches those taxi still in service at current time. Then I add the current tuple (vehicle id, dropoff time) to the list. 

Since carpooling exists in the dataset, there might be multiple tuples having the same vehicle id. We should double count the tuples with the same vehicle id, so I retrieve all the vehicle id, putting them in a set to make them unique. In this way I can count the number of unique vehice id in the list. If the current number of taxi in service is larger than the maxHiredCabs, I will replace the maxHiredCabs with the current numeber.

